In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [36]:
train = r"C:\Users\Admin\Downloads\llm-classification-finetuning\train.csv"
test = r"C:\Users\Admin\Downloads\llm-classification-finetuning\test.csv"

In [37]:
df = pd.read_csv(train)

In [38]:
df.head(4)

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57477 entries, 0 to 57476
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              57477 non-null  int64 
 1   model_a         57477 non-null  object
 2   model_b         57477 non-null  object
 3   prompt          57477 non-null  object
 4   response_a      57477 non-null  object
 5   response_b      57477 non-null  object
 6   winner_model_a  57477 non-null  int64 
 7   winner_model_b  57477 non-null  int64 
 8   winner_tie      57477 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 3.9+ MB


In [40]:
df.isnull().sum()

id                0
model_a           0
model_b           0
prompt            0
response_a        0
response_b        0
winner_model_a    0
winner_model_b    0
winner_tie        0
dtype: int64

In [41]:
df.head(3)

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1


In [42]:
print(df['winner_model_a'].sum())
print(df['winner_model_b'].sum())
print(df['winner_tie'].sum())

20064
19652
17761


In [43]:
#Text Cleaning 
import re
def textCleaning(text):
    return re.sub(r'[^a-zA-Z0-9\s]','',text.lower())
df['prompt'] = df['prompt'].apply(textCleaning)
df['response_a'] = df['response_a'].apply(textCleaning)
df['response_b'] = df['response_b'].apply(textCleaning)

In [44]:
df['prompt_length'] = df['prompt'].apply(len)
df['res_b_length'] = df['response_b'].apply(len)
df['res_a_length'] = df['response_a'].apply(len)

In [45]:
df['prompt_word_count'] = df['prompt'].apply(lambda x:len(x.split()))
df['res_a_word_count'] = df['response_a'].apply(lambda x:len(x.split()))
df['res_b_word_count'] = df['response_b'].apply(lambda x:len(x.split()))

In [46]:
#Encoding Target Variables
df['winner_label'] = df[['winner_model_a','winner_model_b','winner_tie']].idxmax(axis = 1)
df['winner_label']= df['winner_label'].map({
    'winner_model_a':0,
    'winner_model_b':1,
    'winner_tie':2
})

In [47]:
#Preparing Featurues and TArget 
X = df[['res_a_length','res_a_word_count','prompt_length','res_b_length','res_b_word_count','prompt_word_count']]
#target
y = df['winner_label']

In [48]:
print(len(X), len(y))


57477 57477


In [49]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(len(X_train), len(y_train))  # These should match


45981 45981


In [50]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state= 42)
rf.fit(X_train,y_train)

RandomForestClassifier(random_state=42)

In [51]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

y_pred = rf.predict(X_test)
acc_rf = accuracy_score(y_pred,y_test)
acc_rf

0.4261482254697286

Using TF ID,BERT

In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer
combined_data = df['prompt'] + ' '+df['response_a']+' '+df['response_b']
tfid = TfidfVectorizer(max_features=1000)
X_tfid = tfid.fit_transform(combined_data)

In [91]:
#test data 
test_df = pd.read_csv(test)
test_df['prompt'] = test_df['prompt'].apply(textCleaning)
test_df['response_a'] = test_df['response_a'].apply(textCleaning)
test_df['response_b'] = test_df['response_b'].apply(textCleaning)

In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Fit TF-IDF on the training data
tfidf = TfidfVectorizer(max_features=1000)
X_tfid = tfidf.fit_transform(df['prompt'] + ' ' + df['response_a'] + ' ' + df['response_b'])

# Transform the test data using the same vectorizer
Test_tfid = tfidf.transform(test_df['prompt'] + ' ' + test_df['response_a'] + ' ' + test_df['response_b'])

# Combine TF-IDF with additional features for training data
X_combined_train = np.hstack([X_tfid.toarray(),
                              df[['prompt_length', 'res_a_length', 'res_b_length',
                                  'prompt_word_count', 'res_a_word_count', 'res_b_word_count']].values])

# Combine TF-IDF with additional features for test data
X_combined_test = np.hstack([Test_tfid.toarray(),
                             test_df[['prompt_length', 'res_a_length', 'res_b_length',
                                      'prompt_word_count', 'res_a_word_count', 'res_b_word_count']].values])

# Verify shapes
print("Train Combined Shape:", X_combined_train.shape)
print("Test Combined Shape:", X_combined_test.shape)


Train Combined Shape: (57477, 1006)
Test Combined Shape: (3, 1006)


In [92]:
test_df['prompt_length'] = test_df['prompt'].apply(len)
test_df['res_b_length'] = test_df['response_b'].apply(len)
test_df['res_a_length'] = test_df['response_a'].apply(len)
test_df['prompt_word_count'] = test_df['prompt'].apply(lambda x:len(x.split()))
test_df['res_a_word_count'] = test_df['response_a'].apply(lambda x:len(x.split()))
test_df['res_b_word_count'] = test_df['response_b'].apply(lambda x:len(x.split()))

In [105]:
x_train_new, x_val, y_train_new, y_val = train_test_split(
    X_combined_train, y, test_size=0.2, random_state=42
)

In [106]:
rf2 = RandomForestClassifier(random_state=42)
rf2.fit(x_train_new,y_train_new)
y_new_pred = rf2.predict(x_val)
acc_new = accuracy_score(y_new_pred,y_val)
print(f"Accuracy:{acc_new*100}%")

Accuracy:44.48503827418232%


In [107]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier()
dt.fit(x_train_new,y_train_new)
y_dt_pred = dt.predict(x_val)
acc_new_dt = accuracy_score(y_dt_pred,y_val)
print(f"Accuracy:{acc_new_dt*100}%")

Accuracy:38.300278357689635%


In [108]:
from xgboost import XGBClassifier
xg = XGBClassifier()
xg.fit(x_train_new,y_train_new)
y_xg_pred = xg.predict(x_val)
acc_xg = accuracy_score(y_xg_pred,y_val)
print(f"Accuracy:{acc_xg*100}%")

Accuracy:45.45929018789144%


In [109]:
test_df['winner_label'] = xg.predict(X_combined_test)

In [110]:
test_df['winner_label'] = test_df['winner_label'].map({
    0 : 'a',
    1 : 'b',
    2 : 'tie'
})
output = pd.DataFrame({
    'id':test_df['id'],
    'predictions': test_df['winner_label']
})

In [111]:
output.to_csv('submission.csv',index=False)